In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as 
import matplotlib.pyplot as plt
import plotly.express as px
from tensorflow.keras.layers import LSTM, GRU, Conv1D, Bidirectional, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam , SGD , RMSprop
from sklearn.metrics import mean_squared_error

In [ ]:
path = "/content/drive/My Drive/Colab Notebooks/NFLX.csv"
df = pd.read_csv(path)

In [ ]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [ ]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1004,2022-01-31,401.970001,427.700012,398.200012,427.140015,427.140015,20047500
1005,2022-02-01,432.959991,458.480011,425.540009,457.130005,457.130005,22542300
1006,2022-02-02,448.250000,451.980011,426.480011,429.480011,429.480011,14346000
1007,2022-02-03,421.440002,429.260010,404.279999,405.600006,405.600006,9905200
1008,2022-02-04,407.309998,412.769989,396.640015,410.170013,410.170013,7782400


In [ ]:
df.shape

(1009, 7)

In [ ]:
df.Date =  pd.to_datetime(df.Date,format='%Y-%m-%d')

In [ ]:
df.set_index('Date',inplace = True)

In [ ]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [ ]:
data = df[['Close']]

In [ ]:
data.head()

,Close
Date,
2018-02-05,254.259995
2018-02-06,265.720001
2018-02-07,264.559998
2018-02-08,250.100006
2018-02-09,249.470001


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler(feature_range=(0,1))
data_scaled = scalar.fit_transform(data)

In [ ]:
data_scaled

array([[0.04451626],
       [0.06954849],
       [0.06701469],
       ...,
       [0.4272515 ],
       [0.37509011],
       [0.38507243]])

In [ ]:
import math
train_len = math.ceil(len(data)*0.8)
train_len

808

In [ ]:
train = data_scaled[0:train_len]

In [ ]:
test = data_scaled[train_len:]

In [ ]:
X_train=[]
y_train=[] 

for i in range(len(train) - 2):
    j = train[i:(i+2), 0]
    X_train.append(j)
    y_train.append(train[i+2,0])

In [ ]:
X_val=[]
Y_val=[] 

for i in range(2, len(test) - 2):
    j = test[i:(i+2), 0]
    X_val.append(j)
    Y_val.append(test[i + 2, 0])

In [ ]:
X_train, y_train= np.array(X_train), np.array(y_train)
print(X_train.shape , y_train.shape)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape , y_train.shape)

(806, 2) (806,)
(806, 2, 1) (806,)


In [ ]:
X_val, Y_val = np.array(X_val), np.array(Y_val)
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],1))


## **LSTM**

In [ ]:
model=Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mean_squared_error'])
model.summary()
history = model.fit(X_train, y_train, epochs=100 , batch_size = 50,verbose=2, validation_data = (X_val,Y_val))   
#model.fit(X_train, y_train, epochs=50, batch_size=50, verbose=2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 256)               264192    
                                                                 
 dropout_12 (Dropout)        (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 64)                16448     
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dropout_14 (Dropout)        (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                

In [ ]:


lstm_train_pred = model.predict(X_train)
lstm_valid_pred = model.predict(X_val)
print('Train rmse:', np.sqrt(mean_squared_error(y_train, lstm_train_pred)))
print('Validation mse:', mean_squared_error(Y_val, lstm_valid_pred))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_val, lstm_valid_pred)))

7/7 [==============================] - 0s 8ms/step
Train rmse: 0.02468832244770847
Validation mse: 0.001164556938639356
Validation rmse: 0.03412560532268045


In [ ]:
lstm_valid_pred.shape

(197, 1)

# GRU

In [ ]:
model = Sequential()

model.add(GRU(256 , input_shape=(X_train.shape[1],1)))

model.add(Dropout(0.4))
model.add(Dense(64 ,  activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(32))
model.add(Dropout(0.4)) 
model.add(Dense(1))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 256)               198912    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                16448     
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                

In [ ]:
model.compile(loss='mean_squared_error', optimizer=Adam(lr = 0.0005) , metrics = ['mean_squared_error'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(X_train, y_train, epochs=100 , batch_size = 50,verbose=2, validation_data = (X_val,Y_val))   

Epoch 1/100
17/17 - 4s - loss: 0.0649 - mean_squared_error: 0.0649 - val_loss: 0.0197 - val_mean_squared_error: 0.0197 - 4s/epoch - 231ms/step
Epoch 2/100
17/17 - 0s - loss: 0.0211 - mean_squared_error: 0.0211 - val_loss: 0.0462 - val_mean_squared_error: 0.0462 - 230ms/epoch - 14ms/step
Epoch 3/100
17/17 - 0s - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.0082 - val_mean_squared_error: 0.0082 - 236ms/epoch - 14ms/step
Epoch 4/100
17/17 - 0s - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.0063 - val_mean_squared_error: 0.0063 - 227ms/epoch - 13ms/step
Epoch 5/100
17/17 - 0s - loss: 0.0109 - mean_squared_error: 0.0109 - val_loss: 0.0015 - val_mean_squared_error: 0.0015 - 234ms/epoch - 14ms/step
Epoch 6/100
17/17 - 0s - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 0.0050 - val_mean_squared_error: 0.0050 - 229ms/epoch - 13ms/step
Epoch 7/100
17/17 - 0s - loss: 0.0097 - mean_squared_error: 0.0097 - val_loss: 0.0053 - val_mean_squared_error: 0.0053 - 232ms/epoch

In [ ]:

gru_train_pred = model.predict(X_train)
gru_valid_pred = model.predict(X_val)
print('Validation mse:', mean_squared_error(Y_val, gru_valid_pred))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_val, gru_valid_pred)))

7/7 [==============================] - 0s 4ms/step
Validation mse: 0.0021117438300155738
Validation rmse: 0.04595371399588475


## **CNN**

In [93]:
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=1, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(64))
model.add(Dropout(0.4))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam',metrics = ['mean_squared_error'])
history = model.fit(X_train, y_train,epochs=100, verbose=2, validation_data = (X_val,Y_val))

Epoch 1/100
26/26 - 3s - loss: 0.0272 - mean_squared_error: 0.0272 - val_loss: 0.0011 - val_mean_squared_error: 0.0011 - 3s/epoch - 122ms/step
Epoch 2/100
26/26 - 0s - loss: 0.0098 - mean_squared_error: 0.0098 - val_loss: 0.0011 - val_mean_squared_error: 0.0011 - 254ms/epoch - 10ms/step
Epoch 3/100
26/26 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.0024 - val_mean_squared_error: 0.0024 - 212ms/epoch - 8ms/step
Epoch 4/100
26/26 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.0043 - val_mean_squared_error: 0.0043 - 254ms/epoch - 10ms/step
Epoch 5/100
26/26 - 0s - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0034 - val_mean_squared_error: 0.0034 - 158ms/epoch - 6ms/step
Epoch 6/100
26/26 - 0s - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0018 - val_mean_squared_error: 0.0018 - 154ms/epoch - 6ms/step
Epoch 7/100
26/26 - 0s - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0029 - val_mean_squared_error: 0.0029 - 147ms/epoch - 

In [94]:

print(model.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 2, 256)            512       
                                                                 
 dropout_42 (Dropout)        (None, 2, 256)            0         
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dropout_43 (Dropout)        (None, 512)               0         
                                                                 
 dense_36 (Dense)            (None, 64)                32832     
                                                                 
 dropout_44 (Dropout)        (None, 64)                0         
                                                                 
 dense_37 (Dense)            (None, 1)               

In [96]:

cnn_train_pred = model.predict(X_train)
cnn_valid_pred = model.predict(X_val)
print('Train rmse:', np.sqrt(mean_squared_error(y_train, cnn_train_pred)))
print('Validation mse:', mean_squared_error(Y_val, cnn_valid_pred))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_val, cnn_valid_pred)))

7/7 [==============================] - 0s 3ms/step
Train rmse: 0.027113860468140297
Validation mse: 0.0016919329774275464
Validation rmse: 0.04113311290709161
